In [1]:
import numpy as np
import pandas as pd
from utils.load_data import fma_load

features = fma_load('data/fma_metadata/features.csv')

features

tracks = fma_load('data/fma_metadata/tracks.csv')

y = tracks['track','genre_top']

corr = features.corr()

threshold = 0.75

corr_matrix = corr.copy()

np.fill_diagonal(corr_matrix.values, 0) # Diagonal to zero

strong_corr = (corr_matrix.abs() > threshold)

pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if strong_corr.iloc[i, j]:
            pairs.append({
                "feature_1": corr_matrix.columns[i],
                "feature_2": corr_matrix.columns[j],
                "correlation": corr_matrix.iloc[i, j]
            })
pairs



[{'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cens', 'median', '01'),
  'correlation': np.float64(0.9717560508855922)},
 {'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cqt', 'mean', '01'),
  'correlation': np.float64(0.9281310171436797)},
 {'feature_1': ('chroma_cens', 'mean', '01'),
  'feature_2': ('chroma_cqt', 'median', '01'),
  'correlation': np.float64(0.9147936958742171)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cens', 'median', '02'),
  'correlation': np.float64(0.9710105562557106)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'mean', '02'),
  'correlation': np.float64(0.8924722909612783)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'median', '02'),
  'correlation': np.float64(0.891592107193247)},
 {'feature_1': ('chroma_cens', 'mean', '02'),
  'feature_2': ('chroma_cqt', 'skew', '02'),
  'correlation': np.float64(-0.7727008716214067)},
 {'

In [ ]:
features.drop(columns=['chroma_cqt','chroma_cens'],inplace=True)
features = features.loc[:, features.columns.get_level_values('statistics') != 'median']

features

In [ ]:
df_flat = features.copy()
df_flat.columns = ['_'.join(col).strip() for col in df_flat.columns.values]

df_flat['genre_top'] = y

df_flat.dropna(inplace=True)

df_flat

In [ ]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
#import graphviz

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_flat.drop("genre_top",axis=1), df_flat["genre_top"], test_size=0.2, random_state=42, stratify=y)